<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/zerlina-compute-physical/align_template_to_fluorescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- <a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/zerlina-compute-physical/compute_physical.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

<a href="https://github.com/WinetraubLab/coregister-xy/blob/zerlina-compute-physical/compute_physical.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a> -->

# Overview
Use this notebook to get alignment information from ImageJ image registration.

In [32]:
# @title Notebook Inputs { display-mode: "form" }
# @markdown How to use this notebook: [See Instructions](https://docs.google.com/document/d/1G2AME1q6XQhxQ1A2FhkfpktaSFpNXSNQB6mEWwM0YM0/edit?usp=sharing)

# @markdown Input Paths:
# @markdown Leave either path blank to load a file from local file system.
trakem_xml_path = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/alignment.xml" # @param {type:"string"}
fluorescent_patch_number = 8 # @param {type:"integer"}
# @markdown For the alignment of multiple templates to one fluorescent image, specify the patch numbers of each template in the TrakEM stack.
# @markdown Set template_patch_2 to -1 if you are not aligning a second barcode.
template_patch_1 = 11 # @param {type:"integer"}
template_patch_2 = 14 # @param {type:"integer"}

template_size = 401
um_per_pixel = 2

# @markdown Optional: Paths to XML files with exported landmarks. One file per barcode pair. Export each set of selected landmarks.
landmarks_file_1 = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/landmarks1.xml" # @param {type:"string"}
landmarks_file_2 = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/landmarks2.xml" # @param {type:"string"}

landmarks_files = [landmarks_file_1, landmarks_file_2]

import numpy as np
from google.colab import drive
from google.colab import files
drive.mount('/content/drive/')

if not trakem_xml_path:
  print("Upload saved TrakEM project:")
  uploaded = files.upload()
  trakem_xml_path = list(uploaded.keys())[0]
  trakem_xml_path = os.path.join(os.getcwd(), trakem_xml_path)

for idx, f in enumerate(landmarks_files):
  if not f:
    print("Upload saved landmark file", idx+1, ":")
    uploaded = files.upload()
    if uploaded:
      landmarks_f = list(uploaded.keys())[0]
      landmarks_f = os.path.join(os.getcwd(), f)
      landmarks_files.append(landmarks_f)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Upload saved landmark file 2 :


In [ ]:
# @title Environment Setup
!pip install transforms3d
!git clone -b zerlina-compute-physical https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

# from plane.fit_plane import FitPlane
from plane.parse_xml import ParseXML
import matplotlib.pyplot as plt
import os
from google.colab import files
import math

%cd ..

fatal: destination path 'coregister-xy' already exists and is not an empty directory.
/content/coregister-xy
/content


In [9]:
# TEMPORARY-- RUN FOR SETUP WITHOUT GITHUB.
!pip install transforms3d
from parse_xml import ParseXML
import matplotlib.pyplot as plt
import os
from google.colab import files
import math

In [33]:
# @title Compute Physical Transform Parameters, relative to image origin
ps = []
templates = [template_patch_1, template_patch_2]
thetas = []
dist_per_pixel_arr = []
for i in range(0,2):
  if templates[i] == -1:
    continue
  print("Template ", i, " (patch number ", templates[i], "):")
  if len(landmarks_files) > i:
    project = ParseXML.extract_data(trakem_xml_path, fluorescent_patch_number, templates[i], landmarks_files[i], True)
    if landmarks_files[i]:
      error_px = project.find_transformation_error()
      avg_um_px = np.mean([project.calc_real_scale(um_per_pixel, 0), project.calc_real_scale(um_per_pixel, 90)])
      print("Alignment Error: ", "{:.2f}".format(error_px), "px /", "{:.2f}".format(avg_um_px * error_px), "um\n")
    else:
      print()
  else:
    project = ParseXML.extract_data(trakem_xml_path, fluorescent_patch_number, templates[i], None, False)
  ps.append(project)
  tx, ty, theta_deg, sx, sy, shear = project.compute_physical_params()
  print("Center (x,y): ", "{:.2f}".format(tx + template_size/2),", ", "{:.2f}".format(ty + template_size/2), "px")
  print("Rotation (deg): ", "{:.2f}".format(theta_deg))
  thetas.append(theta_deg)
  print("Scale x: ", "{:.2f}".format(sx))
  print("Scale y: ", "{:.2f}".format(sy))
  print("Shear: ", "{:.2f}".format(shear))
  print("\n")
  i += 1

Template  0  (patch number  11 ):
Alignment Error:  4.04 px / 8.37 um

Center (x,y):  821.59 ,  830.84 px
Rotation (deg):  -6.40
Scale x:  0.98
Scale y:  0.95
Shear:  0.01




In [34]:
# @title Calculate Distance and Angle between Barcodes
# Calc centers
if len(ps) > 1:
  center_original = np.array([template_size / 2, template_size / 2, 1])
  transformed_center1 = ps[0].transform_points(center_original)

  transformed_center2 = ps[1].transform_points(center_original)
  # Calc distances
  dist_pixel = np.linalg.norm(transformed_center1 - transformed_center2)
  # Angle
  line_theta_radians = np.arctan2(transformed_center1[1] - transformed_center2[1], transformed_center1[0] - transformed_center2[0])
  line_theta_deg = np.degrees(line_theta_radians)

  for project in ps:
    dist_per_pixel_arr.append(project.calc_real_scale(um_per_pixel, line_theta_deg))

  print("Distance: ", "{:.2f}".format(dist_pixel), "pixels /", "{:.2f}".format(np.mean(dist_per_pixel_arr) * dist_pixel), "um")
  print("Angle (deg): ", "{:.2f}".format(line_theta_deg % 90))
  print("Difference in rotation angle of barcodes (deg): ", "{:.2f}".format(np.abs(thetas[1] - thetas[0])))